In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

import tensorflow_hub as hub

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('train.csv')

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [ ]:
texts = []
for i in range(len(df['Filename'])):
    fn = df['Filename'][i]
    folder = fn.split("-")[0]
    f = open('train-data/'+folder+'/'+fn, "rb")
    texts.append(f.read())
df['text'] = texts

In [ ]:
Outcome = df.copy().drop(['Blinding of intervention','Classes'],axis=1)

In [ ]:
Outcome

In [ ]:
train_labels,train_examples = Outcome['Blinding of Outcome assessment'],Outcome['text'].to_numpy()
train_labels = train_labels.replace(['N','P','Q'],[0,1,2]).to_numpy()

In [ ]:
print("Training entries: {}".format(len(train_examples)))

In [ ]:
train_labels[:10]

In [ ]:
model = ("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")

hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(500, activation='relu'))
model.add(tf.keras.layers.Dense(100))
model.add(tf.keras.layers.Dense(5))

model.summary()

In [11]:

model.compile(
    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.optimizers.Adam(), 
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")])

In [12]:
x_val = train_examples[:100]
partial_x_train = train_examples[100:]

y_val = train_labels[:100]
partial_y_train = train_labels[100:]

NameError: name 'train_examples' is not defined

In [ ]:
history = model.fit(x=partial_x_train, y=partial_y_train,
          validation_data=(x_val, y_val),
          epochs = 30)

In [16]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

NameError: name 'history' is not defined

In [17]:
model.save('./model/outcome.h5')

OSError: Unable to create file (unable to open file: name = './model/outcome.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [18]:
test_predictions = model.predict(train_examples)
test_predictions = np.argmax(test_predictions, axis=-1)

NameError: name 'train_examples' is not defined

In [19]:
result_df = Outcome.copy()
result_df["Predictions"] = test_predictions
result_df["Predictions"] = result_df["Predictions"].replace([0,1,2],['N','P','Q'])

result_df.to_csv(
    "predictionsOut.csv",
    columns=["Predictions","Blinding of Outcome assessment"],
    header=["Predictions","Blinding of Outcome assessment"])

NameError: name 'Outcome' is not defined

In [20]:
cm = tf.math.confusion_matrix(y_val, test_predictions[:100])
cm = cm/cm.numpy().sum(axis=1)[:, tf.newaxis]

NameError: name 'y_val' is not defined

In [21]:
sns.heatmap(
    cm, annot=True)
plt.xlabel("Predicted")
plt.ylabel("True")

NameError: name 'cm' is not defined